In [1]:
import geopandas as gpd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
!cp ../../data/Crops/metadata.geojson ../../DATASET/metadata.geojson

In [3]:
df = gpd.read_file('../../DATASET/metadata.geojson')

In [4]:
df.head()

,Fold,id,ID_PATCH,N_Parcel,Parcel_Cover,TILE,dates-S2,geometry
0,1,lat46024long-416,t30tyr-0,55,0.867065,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.41645 46.02437, -0.39993 46.02399..."
1,3,lat46023long-383,t30tyr-2,53,0.816467,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.38341 46.02361, -0.36689 46.02323..."
2,4,lat46023long-366,t30tyr-3,36,0.598450,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.36689 46.02323, -0.35037 46.02285..."
3,5,lat46022long-350,t30tyr-4,25,0.561584,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.35037 46.02285, -0.33385 46.02247..."
4,1,lat46022long-333,t30tyr-5,28,0.753906,t30tyr,"{'0': '20190906', '1': '20190916', '2': '20191...","POLYGON ((-0.33385 46.02247, -0.31733 46.02208..."


In [38]:
def generate_data_file(row):
    file_in  = '/extra/Sentinel-2-crops/{}/{}/{}.npy'
    file_out = '/projects/Deepleey/agri-tech/sentinel-datasets/LIPS/DATASET/DATA_S2/S2_{}.npy'    
    aux = np.stack([np.load(file_in.format(row['TILE'].upper(), row['id'], v))  for _, v in row['dates-S2'].items()])
    np.save(open(file_out.format(row['ID_PATCH']), 'wb'), aux)
    
def move_target_file(row):
    file_in  = '/extra/Sentinel-2-crops/{}/TARGETS/TARGET_{}.npy'
    file_out = '/projects/Deepleey/agri-tech/sentinel-datasets/LIPS/DATASET/ANNOTATIONS/TARGET_{}.npy'    
    aux = np.load(file_in.format(row['TILE'].upper(), row['id'])) 
    np.save(open(file_out.format(row['ID_PATCH']), 'wb'), aux)

In [39]:
#_ = df.parallel_apply(generate_data_file, axis=1)
_ = df.parallel_apply(move_target_file, axis=1)

In [40]:
!cp -r /projects/Deepleey/agri-tech/sentinel-datasets/LIPS/DATASET/ANNOTATIONS/ /extra/DATASET